<a href="https://colab.research.google.com/github/LokiAndere/After_MIARFID/blob/main/fine_tuning_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

In [ ]:
pretrained_model = ResNet50(
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation='softmax',
)

102973440/102967424 [==============================] - 2s 0us/step


In [ ]:
img_path = '/content/drive/My Drive/posterS.png'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = pretrained_model.predict(x)

# decode the results into a list of tuples (class, description, probability)
print('Predicted:', decode_predictions(preds, top=3)[0])

40960/35363 [==================================] - 0s 0us/step
Predicted: [('n04069434', 'reflex_camera', 0.89795446), ('n04557648', 'water_bottle', 0.013900497), ('n03657121', 'lens_cap', 0.008853288)]


In [ ]:
# h5 file address
#of pretrained model
weights_path = '/content/drive/My Drive/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
# without last layer

WEIGHTS_PATH = ('https://github.com/fchollet/deep-learning-models/'
                'releases/download/v0.2/'
                'resnet50_weights_tf_dim_ordering_tf_kernels.h5')
WEIGHTS_PATH_NO_TOP = ('https://github.com/fchollet/deep-learning-models/'
                       'releases/download/v0.2/'
                       'resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')

#fine image, no interest
#image with deformation, interest
#we could do it with obe class, but for future
#classification let's start with complexity 2
num_classes = 1

fine_tune_model = Sequential()
fine_tune_model.add(
    ResNet50(
        include_top = False,
        weights = 'imagenet',
        #input_shape = (w,h,3),
        pooling = 'avg'
    )
)
fine_tune_model.add( Dense( num_classes, activation = 'softmax' ) )
fine_tune_model.layers[0].trainable = False
print(len(fine_tune_model.layers))

94773248/94765736 [==============================] - 1s 0us/step
2


In [ ]:
fine_tune_model.compile(
    optimizer = 'sgd',
    loss = 'categorical_crossentropy', #log loss
    metrics = [ 'accuracy' ]
)

In [ ]:
data_generator = image.ImageDataGenerator(preprocessing_function = preprocess_input)

In [ ]:
training_data_path = '/content/drive/My Drive/miarfid/data/'
image_size = 256
training_generator = data_generator.flow_from_directory(
    directory = training_data_path + 'train',
    target_size = (image_size, image_size),
    batch_size = 245,
    class_mode = 'categorical'
)
validation_generator = data_generator.flow_from_directory(
    directory = training_data_path + 'validation',
    target_size = (image_size, image_size),
    batch_size = 7,
    class_mode = 'categorical'
)

Found 2450 images belonging to 1 classes.
Found 56 images belonging to 1 classes.


In [ ]:
fine_tune_model.fit_generator(
    generator = training_generator,
    steps_per_epoch = 10,
    validation_data = validation_generator,
    validation_steps = 8
)

TypeError: ignored

In [ ]:
fine_tune_model.fit(
    x = training_generator,
    steps_per_epoch = 10,
    validation_data = validation_generator,
    validation_steps = 8
)

 8/10 [=======================>......] - ETA: 3:27 - loss: 0.0000e+00 - accuracy: 1.0000